# Anlysis of museum collection

import all the module required

In [ ]:
import pandas as pd
import numpy as np

Import the data from the website

In [ ]:
collection_df = pd.read_csv("https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv")
collection_df

We intend to check number of NaNs in all the columns first and try to ingnore those columns which the number of NaNs is no more than 10%.

In [ ]:
collection_df.isna().sum()

In [ ]:
filtered_df =  collection_df.loc[:, collection_df.isna().sum(axis = 0) < len(collection_df.index)*0.1 ]
filtered_df